In [ ]:
#bmd classification

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bmd.csv')
df.head()

,id,age,sex,fracture,weight_kg,height_cm,medication,waiting_time,bmd
0,469,57.052768,F,no fracture,64.0,155.5,Anticonvulsant,18,0.8793
1,8724,75.741225,F,no fracture,78.0,162.0,No medication,56,0.7946
2,6736,70.778900,M,no fracture,73.0,170.5,No medication,10,0.9067
3,24180,78.247175,F,no fracture,60.0,148.0,No medication,14,0.7112
4,17072,54.191877,M,no fracture,55.0,161.0,No medication,20,0.7909


In [ ]:
fracture_mean, no_fracture_mean = df.groupby('fracture')['bmd'].mean()
sigma = np.sqrt((((df['bmd'][df['fracture'] == 'fracture'] - fracture_mean)**2).sum()+
        ((df['bmd'][df['fracture'] == 'no fracture'] - no_fracture_mean)**2).sum())/(len(df)-2))

In [ ]:
freq_table = df.fracture.value_counts()
pr_fracture = freq_table['fracture']/sum(freq_table)
pr_no_fracture = freq_table['no fracture']/sum(freq_table)
pr_fracture, pr_no_fracture

(0.2958579881656805, 0.7041420118343196)

In [ ]:
bmd = 0.54
bmd_fracture = bmd * fracture_mean/(sigma**2) - (fracture_mean**2/(2 * sigma**2) )+ np.log(pr_fracture)
bmd_no_fracture = bmd * no_fracture_mean/(sigma**2) - (no_fracture_mean**2/(2 * sigma**2) )+ np.log(pr_no_fracture)
bmd_fracture,bmd_no_fracture

(7.1345590344847265, 5.381084103346266)

In [ ]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%%R
#libraries that we will need
library(MASS) #lda function
set.seed(1974) #fix the random generator seed 

#read the data
bmd.data     <- 
  read.csv("https://www.dropbox.com/s/c6mhgatkotuze8o/bmd.csv?dl=1", 
           stringsAsFactors = TRUE)
#mean bmd for fracture
mean.f   <- with(bmd.data, mean(bmd[fracture=="fracture"]))  
print('Mean',mean.f)
#mean bmd for no fracture
mean.nf  <- with(bmd.data, mean(bmd[fracture=="no fracture"])) 
#estimate of sigma (see page 141)
sigma.bmd <- sqrt(with(bmd.data,
                       (sum((bmd[fracture=="fracture"] - mean.f)^2) +
                       sum((bmd[fracture=="no fracture"]- mean.nf)^2))/
                    (length(bmd)-2)
                    )
                  )
#probability of fracture/no fracture
pr.fracture <- prop.table(table(bmd.data$fracture))
    
print(c(mean.f, mean.nf, sigma.bmd))
## [1] 0.6233080 0.8502454 0.1305394

print(pr.fracture)
0.54*mean.f/sigma.bmd^2 - mean.f^2/(2*sigma.bmd^2) + log(pr.fracture[1])      

[1] "Mean"
[1] 0.6233080 0.8502454 0.1305394

   fracture no fracture 
   0.295858    0.704142 
fracture 
7.134559 


In [ ]:
ypred = []
for i in range(len(df)):
  val = df['bmd'][i]
  bmd_fracture = bmd * fracture_mean/(sigma**2) - (fracture_mean**2/(2 * sigma**2) )+ np.log(pr_fracture)
  bmd_no_fracture = bmd * no_fracture_mean/(sigma**2) - (no_fracture_mean**2/(2 * sigma**2) )+ np.log(pr_no_fracture)
  if bmd_fracture > bmd_no_fracture:
    ypred.append('fracture')
  else:
    ypred.append('no fracture')

In [ ]:
from sklearn.metrics import confusion_matrix
y = df['fracture']
confusion_matrix(y, ypred)

array([[ 50,   0],
       [119,   0]])

In [ ]:
#Adding more features
feature = ['age','bmd']
mean_fracture = []
mean_no_fracture = []
sigmas = [[0]*len(feature)]*len(feature)
s = []
for i in feature:
  mean1, mean2 = df.groupby('fracture')[i].mean()
  sigma = np.sqrt((((df[i][df['fracture'] == 'fracture'] - mean1)**2).sum()+
        ((df[i][df['fracture'] == 'no fracture'] - mean2)**2).sum())/(len(df)-2))
  mean_fracture.append(mean1)
  mean_no_fracture.append(mean2)
  s.append(sigma)
s

[11.729425411215116, 0.13053936908061506]

In [ ]:
sigmas = [[64.43,0],[0,13]]

In [ ]:
val = [10,0.54]
fracture = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_fracture) - 1/2*np.transpose(mean_fracture).dot(np.linalg.inv(sigmas)).dot(mean_fracture)+pr_fracture
no_fracture = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_no_fracture) - 1/2*np.transpose(mean_no_fracture).dot(np.linalg.inv(sigmas)).dot(mean_no_fracture)+pr_no_fracture
fracture,no_fracture

(-26.64190329980456, -18.738018279545656)

In [ ]:
%%R
library(MASS)
lda.model <- lda(fracture~bmd, data=bmd.data)

#to classify someone with bmd=-.54 
#
predict(lda.model, newdata=data.frame(bmd=0.54))$class
## [1] fracture
## Levels: fracture no fracture
 #predictions
pred.dataset <- predict(lda.model)$class
#confusion matrix
table(bmd.data$fracture, pred.dataset)

             pred.dataset
              fracture no fracture
  fracture          34          16
  no fracture        9         110


In [ ]:
%%R
lda.model2 <- lda(fracture~bmd+age+weight_kg+height_cm,data=bmd.data)
#predictions
pred.dataset2 <- predict(lda.model2)$class
#confusion matrix
table(bmd.data$fracture, pred.dataset2)

#library(irr) #for the kappa statistics
#kappa2(cbind(bmd.data$fracture, pred.dataset))    #model in 1)
# kappa2(cbind(bmd.data$fracture, pred.dataset2))   #current model

             pred.dataset2
              fracture no fracture
  fracture          35          15
  no fracture       11         108


In [ ]:
#bdiag

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bdiag.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
feature = ['texture_mean','radius_mean']
mean_M = []
mean_B = []
sigmas = [[0]*len(feature)]*len(feature)
s=[]
c =0 
for i in feature:
  mean1, mean2 = df.groupby('diagnosis')[i].mean()
  sigma = np.sqrt((((df[i][df['diagnosis'] == 'M'] - mean1)**2).sum()+
        ((df[i][df['diagnosis'] == 'B'] - mean2)**2).sum())/(len(df)-2))
  mean_M.append(mean1)
  mean_B.append(mean2)
  s.append(sigma)
  c+=1
s

[5.385376825786032, 5.845801705237528]

In [ ]:
sigmas = [[5.38,0],[0,5.84]]

In [ ]:
freq_table = df.diagnosis.value_counts()

In [ ]:
freq_table

B    357
M    212
Name: diagnosis, dtype: int64

In [ ]:
pr_M = freq_table['M']/sum(freq_table)
pr_B= freq_table['B']/sum(freq_table)
pr_M,pr_B

(0.37258347978910367, 0.6274165202108963)

In [ ]:
val = [10,20]
Malignant = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_M) - 1/2*np.transpose(mean_M).dot(np.linalg.inv(sigmas)).dot(mean_M)+pr_M
Benign = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_B) - 1/2*np.transpose(mean_B).dot(np.linalg.inv(sigmas)).dot(mean_B)+pr_B
Malignant,Benign

(32.810383964697785, 31.10038369377521)

In [ ]:
ypred = []
for i in range(len(df)):
  val = [df['texture_mean'][i],df['radius_mean'][i]]
  Malignant = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_M) - 1/2*np.transpose(mean_M).dot(np.linalg.inv(sigmas)).dot(mean_M)+pr_M
  Benign = np.transpose(val).dot(np.linalg.inv(sigmas)).dot(mean_B) - 1/2*np.transpose(mean_B).dot(np.linalg.inv(sigmas)).dot(mean_B)+pr_B
  if Malignant > Benign:
    ypred.append('M')
  else:
    ypred.append('B')


In [ ]:
from sklearn.metrics import confusion_matrix
y = df['diagnosis']
confusion_matrix(y, ypred)

array([[ 45, 312],
       [183,  29]])

In [ ]:
%%R
bdiag.data     <- 
  read.csv("/content/drive/MyDrive/bdiag.csv", 
           stringsAsFactors = TRUE)
library(MASS)
lda.model3 <- lda(diagnosis~texture_mean+radius_mean,data=bdiag.data)
#predictions
pred.dataset3 <- predict(lda.model3)$class
#confusion matrix
table(bdiag.data$diagnosis, pred.dataset3)


   pred.dataset3
      B   M
  B 345  12
  M  53 159
